https://snowex-hackweek.github.io/website/tutorials/geospatial/SNOTEL_query.html

In [1]:
import ulmo
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
WSDURL = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'
START = '2004-01-01'
END = '2021-12-31'

In [3]:
# Helper functions
def snotel_swe_fetch(site_code):
    values_df = None
    try: 
        site_values = ulmo.cuahsi.wof.get_values(
            WSDURL, 
            site_code, 
            'SNOTEL:WTEQ_D',
            start=START, 
            end=END)
        values_df = pd.DataFrame.from_dict(site_values['values'])
        values_df = values_df.set_index('datetime')
        values_df.index = pd.to_datetime(values_df.index) 
        
        # Convert values to float and replace -9999 nodata values with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999, np.nan)
        
        # Remove any records flagged with lower quality
        values_df = values_df[values_df['quality_control_level_code'] == '1']
        
        # Add year column
        values_df['year'] =  values_df.index.year
    except:
        print("Unable to fetch SWE for %s" % site_code)
        
    return values_df[['value', 'year']]

def usgs_q_fetch(site_code):
    values_df = pd.DataFrame()
    huc = None
    try:
        site_values = ulmo.usgs.nwis.get_site_data( 
            site_code=site_code,
            service='daily',
            start=START, 
            end=END)
        if site_values:
            if '00060:00003' in list(site_values.keys()):
                values_df = pd.DataFrame.from_dict(site_values['00060:00003']['values'])
                huc = site_values['00060:00003']['site']['huc']
                values_df = values_df.set_index('datetime')
                values_df.index = pd.to_datetime(values_df.index) 
                del values_df['qualifiers']
                values_df = values_df.astype(float)
        
                # Convert values to float and replace -999999 nodata values with NaN
                values_df['value'] = pd.to_numeric(values_df['value']).replace(-999999, np.nan)

                # Add year column
                values_df['year'] =  values_df.index.year

                # Add season column
                values_df['variable'] = values_df.index.to_series().dt.month.map(lambda x: season(x))
                values_df = values_df.loc[values_df['variable']!='Other']
            else:
                print("This gage does not report mean discharge")
        else:
            print('There are no data for this gage')
            return huc, values_df
    except:
        print("Unable to fetch Q for %s" % site_code)
	# # Sum over the season
	# values_df = values_df.groupby(['year', 'season']).sum().unstack()
    return huc, values_df

def season(x):
    if x in range(5, 7):
       return 'Early_Q'
    if x in range(7, 9):
       return 'Late_Q'
    else :
       return 'Other'

In [4]:
# Bring in HUC08s
huc8_path = "/media/nick/Seagate Backup Plus Drive/Data/HUCs/mt_huc8_4326.shp"
huc8 = gpd.read_file(huc8_path)

In [5]:
# Bring in SNOTEL site information
sites = ulmo.cuahsi.wof.get_sites(WSDURL)
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna()

# Filter Montana sites
sites_mt = sites_df.loc[sites_df['site_property'].map(lambda row: row['state']) == 'Montana']

# Convert to GeoDataframe
sites_mt_gdf = sites_mt.copy()
sites_mt_gdf['geometry'] = [Point(float(loc['longitude']), float(loc['latitude'])) for loc in sites_mt['location']]
sites_mt_gdf = gpd.GeoDataFrame(sites_mt_gdf, crs='EPSG:4326')

/home/nick/miniconda3/envs/general/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [6]:
# Find hucs with SNOTEL stations
huc8_snotels = sites_mt_gdf.sjoin(huc8, how="inner")
huc8_snotels_clean = huc8_snotels[['name', 'HUC8']]

In [7]:
# Write and read HUC/SNOTEL key
huc8_path = "./data/huc8_snotels_mt.csv"
huc8_snotels_clean.to_csv(huc8_path)
huc8_snotels_clean = pd.read_csv(huc8_path, index_col=0, dtype={'HUC8': str})
huc8_snotels_clean.index = huc8_snotels.index.set_names(['stat'ion'])
huc8_snotels_clean.reset_index(inplace=True)

In [8]:
# Bring in stream gage information
gages = ulmo.usgs.nwis.get_sites(state_code='MT')
gages_df = pd.DataFrame.from_dict(gages, orient='index').dropna()

# Filter out gages that are in hucs without snotel stations
gages_df = gages_df[gages_df['huc'].isin(huc8_snotels_clean['HUC8'])]

making request for sites: http://waterservices.usgs.gov/nwis/dv/
processing data from request: https://waterservices.usgs.gov/nwis/dv/?format=waterml&stateCd=MT
making request for sites: http://waterservices.usgs.gov/nwis/iv/
processing data from request: https://waterservices.usgs.gov/nwis/iv/?format=waterml&stateCd=MT


In [ ]:
# Process USGS stream gage data
site_codes = gages_df['code'].values 
start_yr = int(START.split('-')[0])
end_yr = int(END.split('-')[0])
q_sum_full = pd.DataFrame()
for site_code in site_codes:
	try:
		huc, q = usgs_q_fetch(site_code)
		if not q.empty :
			q_sum = q.groupby(['year', 'variable']).sum().unstack()
			q_sum.rename({'value': huc}, axis=1, inplace=True)
			q_sum_full = pd.concat([q_sum_full, q_sum.T.reset_index()], ignore_index=True)
			print("Done fetching Q for %s" % site_code)
		else:
			continue
	except Exception as e:
		print(e)
		continue

# Filter out gages with NaN
q_sum_full = q_sum_full.dropna(axis=0)

# Remove column name
q_sum_full.columns.name = None

# Rename HUC column
q_sum_full.rename({'level_0': 'HUC8'}, axis=1, inplace=True)

# Sum values for each HUC and season
q_sum_full = q_sum_full.groupby(['HUC8', 'variable']).sum().reset_index()

In [10]:
# Write and read discharge data to csv
q_path = "data/usgs_mt_huc_early_late_q.csv"
q_sum_full.to_csv(q_path)
q_sum_full = pd.read_csv(q_path, index_col=0, dtype={'HUC8': str})

In [ ]:
# Find annual max SWE for all MT snotel stations
start_yr = int(START.split('-')[0])
end_yr = int(END.split('-')[0])
swe_max_full = pd.DataFrame()
site_codes = sites_mt_gdf.index
for site_code in site_codes:
	try:
		swe = snotel_swe_fetch(site_code)
		swe_max = swe.groupby('year').max()
		swe_max.rename({'value': site_code}, axis=1, inplace=True)
		swe_max_full = pd.concat([swe_max_full, swe_max.T.reset_index()], ignore_index=True)
		# swe_max_full = swe_max_full.merge(swe_max, left_index=True, right_index=True, how='left')
		print("Done fetching SWE for %s" % site_code)
	except:
		continue

# Filter out gages with NaN
swe_max_full = swe_max_full.dropna(axis=0)

# Remove column name
swe_max_full.columns.name = None

# Rename station_id column
swe_max_full.rename({'index': 'station_id'}, axis=1, inplace=True)

# Find HUC associated with each station 
swe_max_full = swe_max_full.merge(
    huc8_snotels_clean, 
    left_on='station_id', 
    right_on='station', 
    how='left')

# Drop unnecessary columns
swe_max_full.drop(['station_id', 'station', 'name'], axis=1, inplace=True)

# Calculate average SWE for each HUC8
swe_max_full = swe_max_full.groupby('HUC8').mean().reset_index()

# Add variable column
swe_max_full['variable'] = np.repeat('SWE_Max', len(swe_max_full))

# Re-order columns and make sure they are strings
swe_cols = swe_max_full.columns.tolist()
swe_cols = [swe_cols[0]] + swe_cols[-1:] + swe_cols[1:-1]
swe_max_full = swe_max_full[swe_cols]
swe_max_full.columns = [str(i) for i in swe_cols]

In [64]:
# Concatenate all data into one DataFrame
final_df = pd.concat([q_sum_full, swe_max_full], ignore_index=True)

# Drop HUCS that do not have all variables
for huc in final_df['HUC8'].unique().tolist():
    nvar = len(final_df[final_df['HUC8'] == huc])
    if nvar < 3:
        final_df.drop(final_df.index[final_df['HUC8'] == huc], inplace=True)

# Sort dataframe by HUC8 and variable        
final_df.sort_values(['HUC8', 'variable'], inplace=True)

In [67]:
# Save final dataset 
swe_path = './data/swe_q_huc_final.csv'
final_df.to_csv(swe_path, index=False)